<h1><center> Aggregated Data Noise Creation <br><br> 
<center> Scikit Mobility and DB Noise<br><br> 
Wendy Shi</center></center>

# Preparation

In [1]:
#Preparation
import pandas as pd
import numpy as np

## Load Count Data

In [4]:
ori_mt = pd.read_csv("IL_Data/Aggregate/IL_ori_agg.csv")
ori_mt = ori_mt.drop(columns = 'Unnamed: 0')
ori_mt.head(3)

,17001,17005,17007,17011,17015,17017,17019,17021,17023,17025,...,17185,17187,17189,17191,17193,17195,17197,17199,17201,17203
0,0.844666,0.000223,0.000160,0.000160,0.000000,0.000511,0.000383,0.000096,0.000351,0.001053,...,0.000096,0.000128,0.000096,0.0000,0.0,0.000638,0.007690,0.002617,0.005074,0.000000
1,0.000133,0.298000,0.000000,0.000000,0.000000,0.001467,0.034000,0.012000,0.000133,0.001733,...,0.000000,0.000267,0.000800,0.0028,0.0,0.003733,0.012400,0.016667,0.001733,0.001067
2,0.000000,0.000000,0.403862,0.000105,0.001679,0.000000,0.002624,0.000000,0.000000,0.000000,...,0.000000,0.004723,0.000000,0.0000,0.0,0.004198,0.021935,0.000000,0.158795,0.000000


In [5]:
column_names = ['S', 'E', 'I', 'R']
pop_2020 = pd.read_csv("../../Data/us_shapefile_county.csv")
pop_2020 = pop_2020[['GEO_ID','pop2020']]
pop_2020["GEO_ID"] = pop_2020["GEO_ID"].apply(lambda x: f"{int(x):05d}")

rename = {'GEO_ID':'GEOID'}
pop_2020 = pop_2020.rename(columns = rename)

print(pop_2020.shape)
pop_2020.tail(3)

(3109, 2)


,GEOID,pop2020
3106,06037,9989165.0
3107,46113,14309.0
3108,51515,6599.0


In [6]:
county_list = ori_mt.columns.tolist()
county_pop = pd.DataFrame()
county_pop['GEOID'] = county_list
county_pop.head(3)

,GEOID
0,17001
1,17005
2,17007


In [8]:
# Get list for the population
joined = pd.merge(county_pop, pop_2020, on = 'GEOID', how = 'left')
pop_list = joined['pop2020'].tolist()
print(f'Length of the population list: {len(pop_list)}')
joined.head()

Length of the population list: 86


,GEOID,pop2020
0,17001,65530.0
1,17005,16663.0
2,17007,53326.0
3,17011,33118.0
4,17015,15664.0


In [9]:
pop_list = joined['pop2020'].tolist()
pop_list[:5]

[65530.0, 16663.0, 53326.0, 33118.0, 15664.0]

In [10]:
ori_count = ori_mt.multiply(pop_list, axis=0)
ori_count.head(3)

,17001,17005,17007,17011,17015,17017,17019,17021,17023,17025,...,17185,17187,17189,17191,17193,17195,17197,17199,17201,17203
0,55350.988545,14.637034,10.455024,10.455024,0.000000,33.456077,25.092058,6.273014,23.001053,69.003159,...,6.273014,8.364019,6.273014,0.0000,0.0,41.820096,503.932161,171.462395,332.469766,0.000000
1,2.221733,4965.574000,0.000000,0.000000,0.000000,24.439067,566.542000,199.956000,2.221733,28.882533,...,0.000000,4.443467,13.330400,46.6564,0.0,62.208533,206.621200,277.716667,28.882533,17.773867
2,0.000000,0.000000,21536.361041,5.596767,89.548279,0.000000,139.919186,0.000000,0.000000,0.000000,...,0.000000,251.854534,0.000000,0.0000,0.0,223.870697,1169.724391,0.000000,8467.909110,0.000000


# Differential Pricavy Noise

In [11]:
#from opendp.trans import make_base_laplace
import opendp.prelude as dp
dp.enable_features("contrib")

In [57]:
count_noise = ori_count.astype(int).copy()
DP_df = pd.DataFrame()

for i in range(86):
    
    if i % 10 == 0: 
        print(i)
        
    epsilon = 0.005 #Privacy budget
    sensitivity = pop_list[i] * 0.0001 # Define sensitivity for each county
    scale = sensitivity / epsilon
    
    # Create the Laplace mechanism
    laplace_mechanism = dp.m.make_laplace(
        dp.atom_domain(T=int), dp.absolute_distance(T=int), scale
    )
    
    def apply_dp_noise(count):
        return laplace_mechanism(count)
    
    new_df_row = count_noise.iloc[[i]].applymap(lambda x: max(laplace_mechanism(x), 0))
    DP_df = pd.concat([DP_df, new_df_row], axis = 0)

0
10
20
30
40
50
60
70
80


In [58]:
#Original
ori_count.head(3)

,17001,17005,17007,17011,17015,17017,17019,17021,17023,17025,...,17185,17187,17189,17191,17193,17195,17197,17199,17201,17203
0,55350.988545,14.637034,10.455024,10.455024,0.000000,33.456077,25.092058,6.273014,23.001053,69.003159,...,6.273014,8.364019,6.273014,0.0000,0.0,41.820096,503.932161,171.462395,332.469766,0.000000
1,2.221733,4965.574000,0.000000,0.000000,0.000000,24.439067,566.542000,199.956000,2.221733,28.882533,...,0.000000,4.443467,13.330400,46.6564,0.0,62.208533,206.621200,277.716667,28.882533,17.773867
2,0.000000,0.000000,21536.361041,5.596767,89.548279,0.000000,139.919186,0.000000,0.000000,0.000000,...,0.000000,251.854534,0.000000,0.0000,0.0,223.870697,1169.724391,0.000000,8467.909110,0.000000


In [59]:
DP_df.head(3)

,17001,17005,17007,17011,17015,17017,17019,17021,17023,17025,...,17185,17187,17189,17191,17193,17195,17197,17199,17201,17203
0,56060,0,0,0,1052,642,3846,0,635,0,...,0,0,527,171,5200,972,586,0,1051,1482
1,0,4821,616,79,614,0,0,0,0,0,...,106,328,329,0,69,135,292,168,172,0
2,0,816,22717,2514,0,1598,0,0,197,0,...,0,0,0,613,47,0,1221,186,9827,696


In [60]:
#Turn count to weight
divisor = [DP_df.iloc[i].sum() for i in range(86)]
noise_mt = DP_df.div(divisor, axis=0)
noise_mt.head(3)

,17001,17005,17007,17011,17015,17017,17019,17021,17023,17025,...,17185,17187,17189,17191,17193,17195,17197,17199,17201,17203
0,0.494723,0.000000,0.000000,0.000000,0.009284,0.005666,0.03394,0.0,0.005604,0.0,...,0.000000,0.000000,0.004651,0.001509,0.045889,0.008578,0.005171,0.000000,0.009275,0.013078
1,0.000000,0.170221,0.021750,0.002789,0.021679,0.000000,0.00000,0.0,0.000000,0.0,...,0.003743,0.011581,0.011616,0.000000,0.002436,0.004767,0.010310,0.005932,0.006073,0.000000
2,0.000000,0.008977,0.249906,0.027656,0.000000,0.017579,0.00000,0.0,0.002167,0.0,...,0.000000,0.000000,0.000000,0.006744,0.000517,0.000000,0.013432,0.002046,0.108105,0.007657


In [61]:
# Calculations
import statistics

#noise_mt_np = ori_mt.to_numpy()
noise_mt_np = noise_mt.to_numpy()
Pii_skmob = [noise_mt_np[i][i] for i in range(86)]


data = Pii_skmob 
mean_value = sum(data) / len(data)  # Mean
median_value = statistics.median(data)  # Median
min_value = min(data)  # Min
max_value = max(data)  # Max

# Output
print(f"Mean: {mean_value}")
print(f"Median: {median_value}")
print(f"Min: {min_value}")
print(f"Max: {max_value}")

Mean: 0.26588393310095537
Median: 0.25843814552666555
Min: 0.04510406692536411
Max: 0.4947227222987045


In [62]:
#noise_mt.to_csv('IL_P05_agg.csv')